In [1]:
import pandas as pd
import email
import os

In [2]:
data_path = "Data/Raw_data"
chinese_path = "trec06c"
english_path = "trec06p"
index_path = "full/index"

In [3]:
labels_ch = pd.read_csv(os.path.join(data_path,chinese_path,index_path), sep=" ", header=None)
labels_en = pd.read_csv(os.path.join(data_path,english_path,index_path), sep=" ", header=None)
labels_ch.columns = ["class", "subpath"]
labels_en.columns = ["class", "subpath"]

In [4]:
def extract_message(row, lang):
    subpath = row["subpath"][3:]
    if lang=="en":
        full_path = os.path.join(data_path, english_path, subpath)
        encode_type = 'ISO-8859-1'
    elif lang=="ch":
        full_path = os.path.join(data_path, chinese_path, subpath)
        encode_type = 'GB2312'
    try:
        with open(full_path, "r", encoding=encode_type) as f:
            email_content = f.read()
            parsed_email_content = email.message_from_string(email_content)
    except:
        row["subject"] = None
        row["email_to"] = None
        row["email_from"] = None
        row["message"] = None
        return row
    row["subject"] = parsed_email_content["subject"]
    row["email_to"] = parsed_email_content["to"]
    row["email_from"] = parsed_email_content["from"]
    if parsed_email_content.is_multipart():
        row["message"] = list(parsed_email_content.get_payload())
    else:
        row["message"] = [parsed_email_content.get_payload()]
    return row

extract_en = lambda row: extract_message(row, lang="en")
extract_ch = lambda row: extract_message(row, lang="ch")

In [5]:
ch_emails = labels_ch.apply(extract_ch, axis=1)

In [6]:
ch_emails.dropna(inplace=True)

In [8]:
ch_emails.to_pickle("ch_emails_raw.pkl")

In [9]:
en_emails = labels_en.apply(extract_en, axis=1)

In [11]:
en_emails.dropna(inplace=True)

In [13]:
en_emails.to_pickle("en_emails_raw.pkl")